In [40]:
import zipfile
path = "/Users/ryo/Downloads/archive.zip"
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('soccer_data')

In [42]:
import sqlite3
import os
print(os.listdir('soccer_data'))
conn = sqlite3.connect('soccer_data/database.sqlite')

['database.sqlite']


In [44]:
import pandas as pd
tables = conn.execute("SELEcT name FROM sqlite_master WHERE type = 'table';").fetchall()
print(tables)

matches = pd.read_sql_query("SEleCT * FROM Match", conn)

teams = pd.read_sql_query("SELECT * From Team", conn)

teams_home = teams.rename(columns = {
    'team_api_id':'home_team_api_id', 
    'team_long_name': 'home_team_name'
})

teams_away = teams.rename(columns = {
    'team_api_id': 'away_team_api_id', 
    'team_long_name': 'away_team_name'
})

matches = matches.merge(teams_home[["home_team_api_id", 'home_team_name']], on = 'home_team_api_id', how = 'left')

matches = matches.merge(teams_away[['away_team_api_id', 'away_team_name']], on = 'away_team_api_id', how = 'left')
pd.set_option('display.max_columns', None)
print(matches.head())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   
3   4           1          1  2008/2009      1  2008-08-17 00:00:00   
4   5           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0        492473              9987              9993               1   
1        492474             10000              9994               0   
2        492475              9984              8635               0   
3        492476              9991              9998               5   
4        492477              7947              9985               1   

   away_team_g

In [46]:
keep_cols = [
    'match_api_id', 'date', 'season', 'home_team_api_id', 'away_team_api_id',
    'home_team_goal', 'away_team_goal', 'home_team_name', 'away_team_name',
    'B365H', 'B365D', 'B365A'
]
matches = matches[keep_cols]
print(matches.head())

   match_api_id                 date     season  home_team_api_id  \
0        492473  2008-08-17 00:00:00  2008/2009              9987   
1        492474  2008-08-16 00:00:00  2008/2009             10000   
2        492475  2008-08-16 00:00:00  2008/2009              9984   
3        492476  2008-08-17 00:00:00  2008/2009              9991   
4        492477  2008-08-16 00:00:00  2008/2009              7947   

   away_team_api_id  home_team_goal  away_team_goal     home_team_name  \
0              9993               1               1           KRC Genk   
1              9994               0               0   SV Zulte-Waregem   
2              8635               0               3  KSV Cercle Brugge   
3              9998               5               0           KAA Gent   
4              9985               1               3      FCV Dender EH   

      away_team_name  B365H  B365D  B365A  
0       Beerschot AC   1.73   3.40   5.00  
1   Sporting Lokeren   1.95   3.20   3.60  
2     RS

In [48]:
matches['data'] = pd.to_datetime(matches['date'])
matches = matches.sort_values('date')

home_stats = matches[['match_api_id', 'date', 'home_team_api_id', 'home_team_goal', 'away_team_goal']]
home_stats = home_stats.rename(columns = {
    'home_team_api_id': 'team_id', 
    'home_team_goal': 'goals_for', 
    'away_team_goal': 'goals_against'
})
home_stats['is_home'] = 1

away_stats = matches[['match_api_id', 'date', 'away_team_api_id', 'home_team_goal', 'away_team_goal']]

away_stats = away_stats.rename(columns = {
    'away_team_api_id': 'team_id', 
    'away_team_goal': 'goals_for',
    'home_team_goal': 'goals_against'
})
away_stats['is_home'] = 0

team_stats = pd.concat([home_stats, away_stats], ignore_index = True)
team_stats = team_stats.sort_values(['team_id', 'date'])


In [52]:
team_stats['avg_goals_for'] = (
    team_stats.groupby('team_id')['goals_for']
    .rolling(5, min_periods = 1).mean()
    .reset_index(0, drop = True)
)
team_stats['avg_goals_against'] = (
    team_stats.groupby('team_id')['goals_against']
    .rolling(5,min_periods = 1).mean()
    .reset_index(0,drop = True)
)
team_stats.head()


,match_api_id,date,team_id,goals_for,goals_against,is_home,avg_goals_for,avg_goals_against
26026,506542,2008-08-10 00:00:00,1601,0,0,0,0.000000,0.000000
85,506549,2008-08-16 00:00:00,1601,2,1,1,1.000000,0.500000
107,506557,2008-08-22 00:00:00,1601,2,1,1,1.333333,0.666667
26163,506567,2008-08-30 00:00:00,1601,0,3,0,1.000000,1.250000
261,506573,2008-09-12 00:00:00,1601,2,0,1,1.200000,1.000000


In [60]:
matches = matches.merge(
    team_stats[["team_id", "date", "avg_goals_for"]],
    left_on = ["home_team_api_id", "date"],
    right_on = ["team_id", "date"],
    how = "left", 
    suffixes = ('', '_home')
)
matches.head()

,match_api_id,date,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_team_name,away_team_name,B365H,B365D,B365A,data,avg_goals_for,team_id,avg_goals_for_home
0,486263,2008-07-18 00:00:00,2008/2009,10192,9931,1,2,BSC Young Boys,FC Basel,NaN,NaN,NaN,2008-07-18,2.0,10192,1.0
1,486264,2008-07-19 00:00:00,2008/2009,9930,10179,3,1,FC Aarau,FC Sion,NaN,NaN,NaN,2008-07-19,1.0,9930,3.0
2,486265,2008-07-20 00:00:00,2008/2009,10199,9824,1,2,FC Luzern,FC Vaduz,NaN,NaN,NaN,2008-07-20,2.0,10199,1.0
3,486266,2008-07-20 00:00:00,2008/2009,7955,10243,1,2,Neuchâtel Xamax,FC Zürich,NaN,NaN,NaN,2008-07-20,2.0,7955,1.0
4,486268,2008-07-23 00:00:00,2008/2009,6493,7955,1,2,AC Bellinzona,Neuchâtel Xamax,NaN,NaN,NaN,2008-07-23,1.5,6493,1.0


In [56]:
matches.columns.tolist()

['match_api_id',
 'date',
 'season',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal',
 'home_team_name',
 'away_team_name',
 'B365H',
 'B365D',
 'B365A',
 'data',
 'team_id',
 'avg_goals_for']

In [58]:
matches = matches.drop(columns = ["team_id"], errors = 'ignore')

In [64]:
def get_result(row):
    if row['home_team_goal'] > row['away_team_goal']:
        return 'home_win'
    elif row['home_team_goal'] < row['away_team_goal']:
        return 'away_win'
    else:
        return 'draw'


matches['result'] = matches.apply(get_result, axis = 1)
matches.head()

,match_api_id,date,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_team_name,away_team_name,B365H,B365D,B365A,data,avg_goals_for,team_id,avg_goals_for_home,result
0,486263,2008-07-18 00:00:00,2008/2009,10192,9931,1,2,BSC Young Boys,FC Basel,NaN,NaN,NaN,2008-07-18,2.0,10192,1.0,away_win
1,486264,2008-07-19 00:00:00,2008/2009,9930,10179,3,1,FC Aarau,FC Sion,NaN,NaN,NaN,2008-07-19,1.0,9930,3.0,home_win
2,486265,2008-07-20 00:00:00,2008/2009,10199,9824,1,2,FC Luzern,FC Vaduz,NaN,NaN,NaN,2008-07-20,2.0,10199,1.0,away_win
3,486266,2008-07-20 00:00:00,2008/2009,7955,10243,1,2,Neuchâtel Xamax,FC Zürich,NaN,NaN,NaN,2008-07-20,2.0,7955,1.0,away_win
4,486268,2008-07-23 00:00:00,2008/2009,6493,7955,1,2,AC Bellinzona,Neuchâtel Xamax,NaN,NaN,NaN,2008-07-23,1.5,6493,1.0,away_win


In [70]:
features = ['avg_goals_for', 'avg_goals_for_home', 'B365H', 'B365D', 'B365A']
X = matches[features]
y = matches['result']

In [72]:
print(X.isnull().sum())

avg_goals_for            0
avg_goals_for_home       0
B365H                 3387
B365D                 3387
B365A                 3387
dtype: int64


In [74]:
X = X.dropna()
y = y.loc[X.index]

In [79]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(list(zip(le.classes_,range(len(le.classes_)))))

[('away_win', 0), ('draw', 1), ('home_win', 2)]


In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size = 0.2, random_state = 42, stratify = y_encoded
)

In [83]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [85]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.5290993582651029
              precision    recall  f1-score   support

    away_win       0.51      0.50      0.51      1301
        draw       0.31      0.23      0.27      1143
    home_win       0.62      0.71      0.66      2075

    accuracy                           0.53      4519
   macro avg       0.48      0.48      0.48      4519
weighted avg       0.51      0.53      0.52      4519



In [89]:
from sklearn.linear_model import LogisticRegression 

model = LogisticRegression(max_iter = 1000, random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.5802168621376411
              precision    recall  f1-score   support

    away_win       0.55      0.62      0.58      1301
        draw       0.39      0.05      0.09      1143
    home_win       0.61      0.85      0.71      2075

    accuracy                           0.58      4519
   macro avg       0.51      0.51      0.46      4519
weighted avg       0.53      0.58      0.51      4519



In [98]:
import xgboost as xgb

model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:41:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.5724717857933171
              precision    recall  f1-score   support

    away_win       0.55      0.58      0.56      1301
        draw       0.38      0.12      0.18      1143
    home_win       0.61      0.82      0.70      2075

    accuracy                           0.57      4519
   macro avg       0.51      0.51      0.48      4519
weighted avg       0.53      0.57      0.53      4519



In [100]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.5182562513830493
              precision    recall  f1-score   support

    away_win       0.48      0.56      0.52      1301
        draw       0.32      0.26      0.29      1143
    home_win       0.64      0.63      0.64      2075

    accuracy                           0.52      4519
   macro avg       0.48      0.48      0.48      4519
weighted avg       0.51      0.52      0.51      4519



In [102]:
from sklearn.linear_model import LogisticRegression 

model = LogisticRegression(class_weight = 'balanced', max_iter = 1000, random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.5563177694180128
              precision    recall  f1-score   support

    away_win       0.55      0.59      0.57      1301
        draw       0.34      0.39      0.36      1143
    home_win       0.71      0.63      0.67      2075

    accuracy                           0.56      4519
   macro avg       0.53      0.54      0.53      4519
weighted avg       0.57      0.56      0.56      4519

